<a href="https://colab.research.google.com/github/dataitgirls4/team_3/blob/miree/%EC%A4%80%EB%8B%A8%EC%96%B4_%ED%86%A0%ED%81%B0%ED%99%94%EB%A5%BC_%ED%86%B5%ED%95%9C_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 열기

네이버 영화평 데이터를 연다.

In [1]:
import pandas as pd


# 파일 열기

In [2]:
nsmc = pd.read_csv('https://github.com/e9t/nsmc/raw/master/ratings_train.txt', sep='\t')


데이터의 초반 5개를 확인한다.

In [3]:
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


데이터의 크기를 확인한다.

In [4]:
nsmc.shape

(150000, 3)

여기서는 2000개만으로 감성분석 실습을 진행한다.

In [5]:
nsmc = nsmc.loc[:1999]

# 준단어 토큰화

인터넷 글들은 맞춤법이나 띄어쓰기를 잘 지키지 않으므로 형태소 분석 대신 `sentencepiece`를 이용해서 준단어 토큰화를 한다.
영화평을 `nsmc.txt` 파일로 저장합니다.

In [7]:
with open('nsmc.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(nsmc['document']))

준단어 토큰을 학습한다. 토큰의 종류는 총 3000개로 한다.

In [10]:
pip install sentencepiece

     |████████████████████████████████| 1.1MB 6.7MB/s 


In [15]:
from sentencepiece import SentencePieceTrainer
SentencePieceTrainer.Train('--input=nsmc.txt --model_prefix=nsmc --vocab_size=3000')

In [14]:
from sentencepiece import SentencePieceProcessor
sp = SentencePieceProcessor()
sp.Load("nsmc.model")

True

In [16]:
sp.encode_as_pieces(nsmc.loc[0, 'document'])

['▁아', '▁더빙', '..', '▁진짜', '▁짜증나', '네요', '▁목소리']

# 단어 문서 행렬(TDM) 만들기

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

옵션 가운데 `lowercase=False`로 설정하면 영어의 대문자를 소문자로 변환을 하지 않는다.


In [18]:
cv = CountVectorizer(lowercase=False, tokenizer=sp.encode_as_pieces)

In [19]:
tdm = cv.fit_transform(nsmc['document'])

# 훈련용 데이터와 테스트용 데이터 나누기

새로운 데이터가 얼마나 일반화되어 있는지 측정하기 위해 훈련용과 테스트용 데이터를 나눈다.
- `x_train` : 훈련용 데이터 (리뷰)
- `y_train` : 훈련용 데이터 (0, 1로 분류된 감정)
- `x_test` : 테스트용 데이터 (리뷰)
- `y_test` : 테스트용 데이터 (0, 1로 분류된 감정)
`sklearn.model_selection`의 `train_test_split` 모듈을 사용한다.



In [20]:
from sklearn.model_selection import train_test_split

In [21]:
x = tdm
y = nsmc['label']

전체 데이터의 20%를 테스트용 데이터로 분할한다.

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 저장

전처리 결과를 저장한다.

In [23]:
import joblib

`cv`에서 `tokenizer`로 사용하고 있는 `sp.encode_as_pieces`는 C++ 언어로 작성된 함수를 Python에서 불러와 사용하는 것으로 `joblib`으로 저장할 수 없다. 따라서 여기서는 빈 값(`None`)으로 바꾸어놓는다.

In [24]:
cv.tokenizer = None

`joblib`으로 저장한다.

In [25]:
joblib.dump((cv, x_train, x_test, y_train, y_test), 'nsmc.pkl')

['nsmc.pkl']